# Финальные выводы

# Итоговый отчёт

## 1. Цель и контекст проекта  
Проект посвящён применению классических методов машинного обучения для анализа химических соединений и прогнозирования их эффективности против вируса гриппа.  
Основные показатели:  
- **IC50** — концентрация, подавляющая активность вируса на 50%.  
- **CC50** — концентрация, вызывающая 50% токсичность для клеток.  
- **SI (Selectivity Index)** — коэффициент избирательности, равный CC50/IC50.

Данные предоставлены в виде Excel-файла с 998 полными записями и ~195 числовыми химическими дескрипторами.

---

## 2. Разведочный анализ данных (EDA)  
- Удалены технические и константные признаки, обработаны пропуски.  
- Распределения IC50, CC50 и SI оказались сильно асимметричны; для наглядности применено логарифмическое преобразование.  
- Корреляции:  
  - IC50 ↔ CC50: ρ ≈ +0.62  
  - IC50 ↔ SI:   ρ ≈ –0.64  
  - CC50 ↔ SI:   ρ ≈ +0.53  
- Ни один дескриптор не коррелирует с целями сильнее |ρ| > 0.3, что указывает на сложный, нелинейный характер зависимостей.

---

## 3. Регрессия

| Задача       | Модель     | R²      | MAE       | MSE            | Комментарий                                                                       |
|--------------|------------|---------|-----------|----------------|-----------------------------------------------------------------------------------|
| **IC50**     | XGBoost    | 0.468   | 158.2     | 109 876.1      | Лучший результат среди моделей; RF и LGBM близки по качеству, LinearRegression не справилась. |
| **CC50**     | XGBoost    | 0.468   | 293.5     | 250 627.2      | Ансамбли деревьев объясняют ≈46% дисперсии; линейная регрессия дала R² ≪ 0.       |
| **SI**       | RandomForest ≈ XGB ≈ LGBM | 0.09    | ~198      | ~1 833 900      | Все модели показали очень низкое качество (R² < 0.1); регрессия SI неэффективна.   |

**Вывод:** регрессия IC50 и CC50 возможна на уровне R²≈0.45–0.50 с помощью XGBoost/LightGBM, но прямой прогноз SI нецелесообразен.

---

## 4. Классификация

### 4.1. IC50 > медианы  
- **RandomForest**: Accuracy 0.740, ROC AUC 0.794  
- **LightGBM**: Accuracy 0.705, ROC AUC 0.775  
- **XGBoost**: Accuracy 0.710, ROC AUC 0.765  
- **LogisticRegression**: Accuracy 0.700, ROC AUC 0.759  

### 4.2. CC50 > медианы  
- **XGBoost**: Accuracy 0.730, ROC AUC 0.800  
- **LightGBM**: Accuracy 0.705, ROC AUC 0.775  
- **RandomForest**: Accuracy 0.690, ROC AUC 0.722  
- **LogisticRegression**: Accuracy 0.690, ROC AUC 0.689  

### 4.3. SI > медианы  
- **LightGBM**: Accuracy 0.635, ROC AUC 0.675  
- **RandomForest**: Accuracy 0.630, ROC AUC 0.675  
- **LogisticRegression**: Accuracy 0.620, ROC AUC 0.665  
- **XGBoost**: Accuracy 0.610, ROC AUC 0.649  

### 4.4. SI > 8  
- **LightGBM**: Accuracy 0.730, F1 0.614, ROC AUC 0.724  
- **RandomForest**: Accuracy 0.690, F1 0.530, ROC AUC 0.722  
- **XGBoost**: Accuracy 0.705, F1 0.587, ROC AUC 0.727  
- **LogisticRegression**: Accuracy 0.690, F1 0.569, ROC AUC 0.689  

**Вывод:**  
- Для сбалансированных задач (IC50 и CC50) и муляжного порога SI>медианы точность классификации достигает 63–74%, ROC AUC ≈0.68–0.80.  
- В задаче жёсткого порога SI>8 модели достигают **Accuracy до 73%**, **ROC AUC до 0.73**; LightGBM демонстрирует лучший F1.  
- Градиентный бустинг (LightGBM, XGBoost) и случайный лес — лучшие методы для всех классификационных задач.

---

## 5. Ключевые химические дескрипторы  
Анализ feature importance показал, что для всех задач наиболее информативными оказались:  
- **SlogP_VSAx** (поверхностные дескрипторы, связанные с липофильностью)  
- **PEOE_VSAx** (электростатические поверхности)  
- **EState индексы** (электротопологические характеристики)  
- **Количество ароматических колец** и **полярная площадь поверхности (TPSA)**  

---

## 6. Общий вывод и рекомендации  
1. **XGBoost и LightGBM** — универсальные, надёжные решения для задач регрессии (IC50, CC50) и классификации.  
2. **Прямая регрессия SI** нецелесообразна из-за низкой объясняющей способности; вместо этого лучше использовать **классификацию** по порогам.  
3. Для **подборки лидов** оптимально применять модель классификации SI > 8, чтобы отобрать наиболее селективные соединения.  
4. При дальнейшей работе можно добавить:  
   - **Feature engineering** (создание новых дескрипторов),  
   - **Ансамблирование** разных алгоритмов,  
   - **Тщательный подбор гиперпараметров** (Optuna),  
   - **Анализ нестандартных признаков** (метаболические/биофизические данные).  

Проект демонстрирует, что классические методы ML позволяют существенно ускорить приоритизацию химических кандидатов и снизить затраты на эксперименты.  
